In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sapcar_daily
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
 AS
WITH
sapcar_full_inventory AS (
    SELECT * 
    FROM {catalog_name}.udp_wcm_silver_sap_car.sapcar_full_inventory 
    WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) 
      AND calhour < 7
),
total AS (
    SELECT DISTINCT 
        FARM_FINGERPRINT(CONCAT(
            IFNULL(CAST(IF(c.store_id IS NOT NULL AND a.calday >= c.cutoff_date, c.store_id, a.store_id) AS STRING), ""),
            IFNULL(CAST(a.product_id AS STRING), ""),
            IFNULL(CAST(a.calday AS STRING), "")
        )) AS hash_id,
        IF(c.store_id IS NOT NULL AND a.calday >= c.cutoff_date, c.store_id, a.store_id) AS store_id,
        CAST(a.product_id AS STRING) AS product_id,
        a.calday,
        a.calhour,
        a.closing_stock_qty
    FROM sapcar_full_inventory a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_franchise_config c 
        ON CAST(a.store_id AS STRING) = c.ref_fc_store

    UNION ALL

    SELECT 
        FARM_FINGERPRINT(CONCAT(
            IFNULL(CAST(a.store_id AS STRING), ""),
            IFNULL(CAST(a.article_id AS STRING), ""),
            IFNULL(CAST(a.calday AS STRING), "")
        )) AS hash_id,
        a.store_id,
        CAST(a.article_id AS STRING) AS product_id,
        a.calday,
        NULL AS calhour,
        a.stock_qty AS closing_stock_qty
    FROM {catalog_name}.udp_wcm_silver_rop.fms_skustock a  --franchise_skustock
    
    WHERE REGEXP_LIKE(a.store_id, r'F[0-9]')
      AND a.calday =  DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) 
      AND a.stock_qty IS NOT NULL
)
SELECT
    hash_id,
    store_id,
    product_id,
    calday,
    calhour,
    CAST(SUM(closing_stock_qty) AS DOUBLE) AS closing_stock_qty
FROM total
GROUP BY hash_id, calday, calhour, store_id, product_id
""")